In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import subprocess, cv2, json, os, sys, shutil, pyk4a, time
import numpy as np, matplotlib.pyplot as plt
from kinectacq.acquisition import start_recording
from kinectacq.paths import DATA_DIR, ensure_dir

### Set up recording location and duration info

In [3]:
import datetime

In [4]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
timestamp

'2022-02-18_13-10-00'

In [5]:
filename_prefix = DATA_DIR / 'test_recording' / timestamp
recording_length = 10

In [6]:
ensure_dir(filename_prefix)

In [7]:
filename_prefix

PosixPath('/n/groups/datta/tim_sainburg/projects/kinectacq/data/test_recording/2022-02-18_13-10-00')

### Get camera information

In [8]:
!k4arecorder --list

Index:0	Serial:000774310512	Color:1.6.102	Depth:1.6.75
Index:1	Serial:000261501812	Color:1.6.108	Depth:1.6.79


In [9]:
import datetime, subprocess, numpy as np, cv2, time, sys
from multiprocessing import Process, Queue
from pyk4a import *



def get_number_of_frames(filepath):
    command = 'ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames -of default=nokey=1:noprint_wrappers=1'
    out = subprocess.Popen(command.split(' ')+[filepath], 
               stdout=subprocess.PIPE, 
               stderr=subprocess.STDOUT)
    stdout,stderr = out.communicate()    
    return int(stdout.decode('utf8').strip('\n'))


def write_frames(filename, frames, threads=6, fps=30, crf=10,
                 pixel_format='gray8', codec='h264', close_pipe=True,
                 pipe=None, slices=24, slicecrc=1, frame_size=None, get_cmd=False):
    """
    Write frames to avi file using the ffv1 lossless encoder
    """

    # we probably want to include a warning about multiples of 32 for videos
    # (then we can use pyav and some speedier tools)

    if not frame_size and type(frames) is np.ndarray:
        frame_size = '{0:d}x{1:d}'.format(frames.shape[2], frames.shape[1])

    command = ['ffmpeg',
               '-y',
               '-loglevel', 'fatal',
               '-framerate', str(fps),
               '-f', 'rawvideo',
               '-s', frame_size,
               '-pix_fmt', pixel_format,
               '-i', '-',
               '-an',
               '-crf',str(crf),
               '-vcodec', codec,
               '-preset', 'ultrafast',
               '-threads', str(threads),
               '-slices', str(slices),
               '-slicecrc', str(slicecrc),
               '-r', str(fps),
               filename]

    if get_cmd:
        return command

    if not pipe:
        pipe = subprocess.Popen(
            command, stdin=subprocess.PIPE, stderr=subprocess.PIPE)

    for i in range(frames.shape[0]):
        pipe.stdin.write(frames[i,:,:].astype('uint8').tobytes())

    if close_pipe:
        pipe.stdin.close()
        return None
    else:
        return pipe


def write_color_frames(filename, frames, threads=6, fps=30, crf=22,
                 pixel_format='rgb24', codec='h264',close_pipe=True,
                 pipe=None,  slices=24, slicecrc=1, frame_size=None, get_cmd=False):
    """
    Write frames to avi file using the ffv1 lossless encoder
    """

    # we probably want to include a warning about multiples of 32 for videos
    # (then we can use pyav and some speedier tools)

    if not frame_size and type(frames) is np.ndarray:
        frame_size = '{0:d}x{1:d}'.format(frames.shape[2], frames.shape[1])

    command = ['ffmpeg',
               '-y',
               '-loglevel', 'fatal',
               '-threads', str(threads),
               '-framerate', str(fps),
               '-f', 'rawvideo',
               '-s', frame_size,
               '-pix_fmt', pixel_format,
               '-i', '-',
               '-an',
               '-vcodec', codec,
               '-preset', 'ultrafast',
               '-slices', str(slices),
               '-slicecrc', str(slicecrc),
               '-r', str(fps),
               '-crf',str(crf),
               filename]


    if get_cmd:
        return command

    if not pipe:
        pipe = subprocess.Popen(
            command, stdin=subprocess.PIPE, stderr=subprocess.PIPE)

    for i in range(frames.shape[0]):
        pipe.stdin.write(frames[i,:,:,:].astype('uint8').tobytes())

    if close_pipe:
        pipe.stdin.close()
        return None
    else:
        return pipe

    

def read_frames(filename, frames, threads=6, fps=30,
                pixel_format='gray8', frame_size=(640,576),
                slices=24, slicecrc=1, get_cmd=False):
    """
    Reads in frames from the .mp4/.avi file using a pipe from ffmpeg.
    Args:
        filename (str): filename to get frames from
        frames (list or 1d numpy array): list of frames to grab
        threads (int): number of threads to use for decode
        fps (int): frame rate of camera in Hz
        pixel_format (str): ffmpeg pixel format of data
        frame_size (str): wxh frame size in pixels
        slices (int): number of slices to use for decode
        slicecrc (int): check integrity of slices
    Returns:
        3d numpy array:  frames x h x w
    """

    command = [
        'ffmpeg',
        '-loglevel', 'fatal',
        '-ss', str(datetime.timedelta(seconds=frames[0]/fps)),
        '-i', filename,
        '-vframes', str(len(frames)),
        '-f', 'image2pipe',
        '-s', '{:d}x{:d}'.format(frame_size[0], frame_size[1]),
        '-pix_fmt', pixel_format,
        '-threads', str(threads),
        '-slices', str(slices),
        '-slicecrc', str(slicecrc),
        '-vcodec', 'rawvideo',
        '-'
    ]

    if get_cmd:
        return command

    pipe = subprocess.Popen(command, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    out, err = pipe.communicate()
    if(err):
        print('error', err)
        return None
    video = np.frombuffer(out, dtype='uint8').reshape((len(frames), frame_size[1], frame_size[0]))
    return video


def read_color_frames(filename, frames, threads=6, fps=30,
                pixel_format='rgb24', frame_size=(640,576),
                slices=24, slicecrc=1, get_cmd=False):
    """
    Reads in frames from the .mp4/.avi file using a pipe from ffmpeg.
    Args:
        filename (str): filename to get frames from
        frames (list or 1d numpy array): list of frames to grab
        threads (int): number of threads to use for decode
        fps (int): frame rate of camera in Hz
        pixel_format (str): ffmpeg pixel format of data
        frame_size (str): wxh frame size in pixels
        slices (int): number of slices to use for decode
        slicecrc (int): check integrity of slices
    Returns:
        3d numpy array:  frames x h x w
    """

    command = [
        'ffmpeg',
        '-loglevel', 'fatal',
        '-ss', str(datetime.timedelta(seconds=frames[0]/fps)),
        '-i', filename,
        '-vframes', str(len(frames)),
        '-f', 'image2pipe',
        '-s', '{:d}x{:d}'.format(frame_size[0], frame_size[1]),
        '-pix_fmt', pixel_format,
        '-threads', str(threads),
        '-slices', str(slices),
        '-slicecrc', str(slicecrc),
        '-vcodec', 'rawvideo',
        '-'
    ]

    if get_cmd:
        return command

    pipe = subprocess.Popen(command, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    out, err = pipe.communicate()
    if(err):
        print('error', err)
        return None
    video = np.frombuffer(out, dtype='uint8').reshape((len(frames), frame_size[1], frame_size[0],3))
    return video





def display_images(display_queue):
    while True: 
        data = display_queue.get() 
        if len(data)==0: 
            cv2.destroyAllWindows()
            break
        else:
            ir = data[0]
            cv2.imshow('ir',ir)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 


def write_images(image_queue, filename_prefix):
    depth_pipe = None
    ir_pipe = None 
    start_time = time.time()
    while True: 
        data = image_queue.get() 
        if len(data)==0: 
            depth_pipe.stdin.close()
            ir_pipe.stdin.close()
            break
        else:
            ir,depth = data
            print('frame',time.time() - start_time)
            depth_pipe = write_frames(filename_prefix+'.depth.avi', depth.astype(np.uint8)[None,:,:], codec='ffv1', close_pipe=False, pipe=depth_pipe)
            ir_pipe = write_frames(filename_prefix+'.ir.avi', ir.astype(np.uint8)[None,:,:], close_pipe=False, crf=14, pipe=ir_pipe)

  

def capture_from_azure(k4a, filename_prefix, recording_length, display_frames=False, display_time=False):
    
    image_queue = Queue()
    write_process = Process(target=write_images, args=(image_queue, filename_prefix))
    write_process.start()
    
    if display_frames: 
        display_queue = Queue()
        display_process = Process(target=display_images, args=(display_queue,))
        display_process.start()
        
    k4a.start()
    system_timestamps = []
    device_timestamps = []
    start_time = time.time()
    count = 0
    
    try:
        while time.time()-start_time < recording_length:  
            capture = k4a.get_capture()
            if capture.depth is None: 
                print('Dropped frame')
                continue
            
            system_timestamps.append(time.time())
            device_timestamps.append(capture.depth_timestamp_usec)

            depth = capture.depth.astype(np.int16)
            ir = capture.ir.astype(np.uint16)

            depth = np.clip((depth-435) * (depth < 690), 0, 255).astype(np.uint8)
            ir = np.clip(ir+100,160,5500)
            ir = ((np.log(ir)-5)*70).astype(np.uint8)

            image_queue.put((ir,depth))
            if display_frames and count % 2 == 0: 
                display_queue.put((ir[::2,::2],))

            if display_time and count % 15 == 0: 
                sys.stdout.write('\rRecorded '+repr(int(time.time()-start_time))+' out of '+repr(recording_length)+' seconds')
            count += 1
            
    except OSError:
        print('Recording stopped early')
        
    finally:
        k4a.stop()
        system_timestamps = np.array(system_timestamps) 
        np.save(filename_prefix+'.system_timestamps.npy',system_timestamps)
        np.save(filename_prefix+'.device_timestamps.npy',device_timestamps)
        print(' - Frame rate = ',len(system_timestamps) / (system_timestamps.max()-system_timestamps.min()))

        image_queue.put(tuple())
        write_process.join()

        if display_frames:
            display_queue.put(tuple())
            display_process.join()
            

def start_recording(filename_prefix, recording_length,top_device_id=0, bottom_device_id=1, display='top'):
    
    k4a_bottom = PyK4A(Config(color_resolution=ColorResolution.RES_720P,
                          depth_mode=DepthMode.NFOV_UNBINNED,
                          synchronized_images_only=False,
                          wired_sync_mode=WiredSyncMode.MASTER), device_id=bottom_device_id)

    k4a_top    = PyK4A(Config(color_resolution=ColorResolution.OFF,
                              depth_mode=DepthMode.NFOV_UNBINNED,
                              synchronized_images_only=False,
                              wired_sync_mode=WiredSyncMode.SUBORDINATE,
                              subordinate_delay_off_master_usec=640), device_id=top_device_id)

    p_top    = Process(target=capture_from_azure,
                       args=(k4a_top, filename_prefix+'.top', recording_length),
                       kwargs={'display_frames': display=='top', 'display_time': display!='top'})

    p_bottom = Process(target=capture_from_azure, 
                       args=(k4a_bottom, filename_prefix+'.bottom' , recording_length),
                       kwargs={'display_frames': display!='top', 'display_time': display=='top'})

    p_top.start()
    p_bottom.start()
    
    
    
    
def save_camera_params(prefix, bottom_device_id=0, top_device_id=1):
    k4a_bottom = PyK4A(Config(color_resolution=ColorResolution.RES_720P,
                              depth_mode=DepthMode.NFOV_UNBINNED,
                              synchronized_images_only=False,
                              wired_sync_mode=WiredSyncMode.MASTER), device_id=bottom_device_id)

    k4a_top    = PyK4A(Config(color_resolution=ColorResolution.OFF,
                              depth_mode=DepthMode.NFOV_UNBINNED,
                              synchronized_images_only=False,
                              wired_sync_mode=WiredSyncMode.SUBORDINATE,
                              subordinate_delay_off_master_usec=640), device_id=top_device_id)


    k4a_top.start()
    k4a_bottom.start()
    time.sleep(1)
    k4a_bottom.save_calibration_json(prefix+'.bottom.json')
    k4a_top.save_calibration_json(prefix+'.top.json')
    k4a_top.stop()
    k4a_bottom.stop()


In [10]:
filename_prefix = 'data/222_2_2_GRIN61_MeA8A_training'
recording_length = 10

filename_prefix += '.rig2'

In [11]:
ensure_dir(filename_prefix)

In [12]:
 start_recording(filename_prefix, recording_length,top_device_id=1, bottom_device_id=0, display='bottom')

Recorded 0 out of 10 secondsframe 0.8343043327331543
frame 0.8840956687927246
Recorded 9 out of 10 seconds - Frame rate =  30.175553158846498
 - Frame rate =  30.242071858536598
frame 22.80235481262207
frame 22.84596347808838
frame 22.84973931312561
frame 22.858043670654297
frame 22.864826202392578
frame 22.878883838653564
frame 22.890223503112793
frame 22.901337385177612
frame 22.909562826156616
frame 22.91334104537964
frame 22.91958475112915
frame 22.93820095062256
frame 22.948480129241943
frame 22.95902681350708
frame 22.967909812927246
frame 22.978976726531982
frame 22.992884635925293
frame 23.004437923431396
frame 23.020030736923218
frame 23.021924257278442
frame 23.027828454971313
frame 23.042885780334473
frame 23.050959587097168
frame 23.05823564529419
frame 23.072895526885986
frameframe 23.07877016067505
 23.087290287017822
frameframe  23.09702134132385323.087558269500732

frame 23.10904622077942frame
 23.111359357833862frame 23.118780851364136

frameframe 23.128732681274414 
f

frame25.274742126464844
frame  25.2886383533477825.287662267684937

frame frame25.30438756942749
 frame 25.30610775947570825.314597368240356
frame
 25.321682691574097
frame frame25.323029041290283 25.33307385444641

frameframe 25.34728693962097
 frame25.340765953063965
 frame25.354656219482422 
25.358718633651733
frame 25.365766525268555frame
 25.375455617904663
frame frame25.38044571876526 25.389100790023804
frame 
25.397119998931885
frameframe 25.40039587020874
 frame25.408031463623047 25.410757780075073

frameframe  25.42832756042480525.421473741531372

frame frame 25.4341506958007825.441955089569092
frame 25.446924209594727
frame 
25.451632738113403
frame 25.463762998580933
frame 25.464908123016357
frame frame 25.47127032279968325.478702068328857

frameframe  25.4870018959045425.490530014038086
frame
 25.497284173965454frame
 25.49739098548889
frame 25.508698225021362
frameframe  25.514901161193848
25.517441511154175frame 
frame25.52568030357361 
25.53713583946228
frameframe  25.55